## 1D Convolution 

In [2]:
import torch
import torch.nn as nn

In [8]:
# Create a sample torch matrix 

matrix = torch.tensor([[2, 1], [5, 2], [1, 0], [4, 6], [3, 8]], dtype=torch.float32)

kernel = torch.tensor([[1, 0], [0, 1], [2, 5]], dtype=torch.float32)

In [9]:
print(matrix.shape)
print(kernel.shape)

torch.Size([5, 2])
torch.Size([3, 2])


In [16]:
print(matrix)
print(kernel)

tensor([[2., 1.],
        [5., 2.],
        [1., 0.],
        [4., 6.],
        [3., 8.]])
tensor([[1., 0.],
        [0., 1.],
        [2., 5.]])


In [22]:
# iterating through matrix, through axis 0 (rows) 
for i in range(matrix.shape[0]): 
    print(matrix[i, :])

print(torch.norm(matrix[0, :]))

tensor([2., 1.])
tensor([5., 2.])
tensor([1., 0.])
tensor([4., 6.])
tensor([3., 8.])
tensor(2.2361)


In [19]:
# example calculations 
a = torch.tensor([1, 2, 3], dtype=torch.float32)
b = torch.tensor([4, 1, 6], dtype=torch.float32)

print(b - a)

result = torch.norm(b)
print(result)

tensor([ 3., -1.,  3.])


In [56]:
# getting the minimum value of a tensor
matrix = torch.tensor([[2, 1], [5, 2], [1, 0], [4, 6], [3, 8]], dtype=torch.float32)
print(matrix)

min_value_row = torch.topk(matrix[:,0], 3, largest=False).values
print(min_value_row)

min_indicies_row = torch.topk(matrix[:,0], 3, largest=False).indices
print(min_indicies_row)

print(matrix[0, :])

tensor([[2., 1.],
        [5., 2.],
        [1., 0.],
        [4., 6.],
        [3., 8.]])
tensor([1., 2., 3.])
tensor([2, 0, 4])
tensor([2., 1.])


In [59]:
min_rows = matrix[min_indicies_row]
print(min_rows.shape)

tensor_list = [min_rows, min_rows]
cat = torch.cat(tensor_list, dim=0)
print(cat.shape)
print(cat)


torch.Size([3, 2])
torch.Size([6, 2])
tensor([[1., 0.],
        [2., 1.],
        [3., 8.],
        [1., 0.],
        [2., 1.],
        [3., 8.]])


In [46]:

# Create a list of tensors
tensor_list = [torch.tensor([1, 2, 3]), torch.tensor([4, 5, 6]), torch.tensor([7, 8, 9])]

# Stack the tensors along a new dimension
stacked_tensors = torch.stack(tensor_list)

# Sum the stacked tensors along the appropriate dimension
summed_tensors = torch.sum(stacked_tensors, dim=0)

print(stacked_tensors)

tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])


## Example object + convolution 2d & 1d

In [2]:
''' Nearest neighbor search with Pytorch Tensors '''
import torch


    
class NNT: # Nearest Neighbor Tensor
    def __init__(self, matrix, kernel): 
        self._matrix = matrix 
        self._kernel = kernel

        # Calculate the distance matrix
        self._dist_matrix = torch.zeros(matrix.shape[0], matrix.shape[0])
        for i in range(matrix.shape[0]):
            for j in range(matrix.shape[0]):
                self._dist_matrix[i, j] = torch.norm(matrix[i] - matrix[j])

        self._num_closest = int(kernel.shape[0])
        self._convolution_matrix = self.calc_convolution_matrix()

    @property 
    def matrix(self): 
        '''Returns the matrix of the NNT object'''
        return self._matrix
    
    @property 
    def kernel(self): 
        '''Returns the kernel of the NNT object'''
        return self._kernel
    
    @property 
    def dist_matrix(self): 
        '''Returns the distance matrix of the NNT object'''
        return self._dist_matrix
    

    @property 
    def num_closest(self): 
        '''Returns the number of closest neighbors to be used in the convolution matrix'''
        return self._num_closest
        
    
    @property 
    def convolution_matrix(self): 
        '''Returns the convolution matrix of the NNT object'''
        return self._convolution_matrix
    
    @matrix.setter 
    def matrix(self, value): 
        # Check if the matrix is a torch.Tensor
        if not isinstance(value, torch.Tensor): 
            raise ValueError("Matrix must be a torch.Tensor")
        self._matrix = value
        

                
                
    @kernel.setter
    def kernel(self, value): 
        # Check if the kernel is a torch.Tensor
        if not isinstance(value, torch.Tensor): 
            raise ValueError("Kernel must be a torch.Tensor")
        
        # No Errors 
        self._kernel = value
        
    @num_closest.setter
    def num_closest(self, value): 
        # Check if the number of closest neighbors is less than the number of rows in the matrix
        if value > self._matrix.shape[0]:
            raise ValueError("Number of closest neighbors cannot exceed the number of rows in the matrix")
        self._num_closest = value
        
    def calc_convolution_matrix(self): 
        tensor_list = [] 
        for i in range(self._matrix.shape[0]): 
            # Get the indices of the closest neighbors from the distance matrix
            min_indicies = torch.topk(self._dist_matrix[:, i], self.num_closest, largest=False).indices
            
            # Get the rows of the matrix that correspond to the closest neighbors
            min_rows = self._matrix[min_indicies]
            
            # Append the rows to the tensor list for later concatenation 
            tensor_list.append(min_rows)
            
        # Concatenate the tensor list to create the convolution matrix
        return torch.cat(tensor_list, dim=0)
            
            
        

        


In [3]:
ex_matrix = torch.tensor([[2, 1], [5, 2], [1, 0], [4, 6], [3, 8]], dtype=torch.float32)
ex_kernel = torch.tensor([[1, 0], [0, 1], [2, 5]], dtype=torch.float32)

example = NNT(ex_matrix, ex_kernel)


In [4]:
example.convolution_matrix.shape


torch.Size([15, 2])

In [6]:
a = example.convolution_matrix.unsqueeze(0).unsqueeze(1)

print(a.shape)
print(a)


torch.Size([1, 1, 15, 2])
tensor([[[[2., 1.],
          [1., 0.],
          [5., 2.],
          [5., 2.],
          [2., 1.],
          [4., 6.],
          [1., 0.],
          [2., 1.],
          [5., 2.],
          [4., 6.],
          [3., 8.],
          [5., 2.],
          [3., 8.],
          [4., 6.],
          [5., 2.]]]])


In [12]:
# using the Conv2d layer in Pytorch
import torch 
from torch import nn 

# create filter 

a = example.convolution_matrix.unsqueeze(0).unsqueeze(1)

filter = torch.rand(1, 3, 2) 

conv2d_layer = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=(3, 2), stride = 3)

a2 = conv2d_layer(a)

print(a2.shape)
print(a2)

print(conv2d_layer.weight)

torch.Size([1, 1, 5, 1])
tensor([[[[-1.7648],
          [-3.9012],
          [-1.6820],
          [-3.1558],
          [-2.5407]]]], grad_fn=<ConvolutionBackward0>)
Parameter containing:
tensor([[[[-0.2990,  0.0542],
          [-0.0388, -0.1233],
          [-0.0707, -0.3005]]]], requires_grad=True)


In [10]:
# using Conv1d layer in Pytorch 
import torch 
from torch import nn 
b = example.convolution_matrix.unsqueeze(1)

filter = torch.randn(1, 3, 2) # 1 output channel, 3x2 filter size

conv1d_layer = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=3, stride = 3)

b2 = conv1d_layer(b)

print(b2.shape)

RuntimeError: Calculated padded input size per channel: (2). Kernel size: (3). Kernel size can't be greater than actual input size